# HowTo-1-DownloadingImageCutouts

At first we will try to import the qso_toolbox modules:

In [1]:
from qso_toolbox import catalog_tools as ct, image_tools as it, photometry_tools as pt, utils as ut

## (1) The examples files 
We have prepared example tables to test the qso_toolbox modules with: 

### A) stripe82_milliquas_190210.hdf5 
(stripe82_milliquas_190210.csv, stripe82_milliquas_190210.fits)

This file contains all confirmed quasars within the Milliquas catalog (from Feb 10th 2019, intermediate release; https://heasarc.gsfc.nasa.gov/W3Browse/all/milliquas.html), which fall within the Stripe 82 footprint. This file includes the following columns: [RA:'mq_ra', Dec:'mq_dec, Name:'mq_name', Redshift:'mq_z', Reference:'mq_cite'].

### The example below shows how to read these files into a pandas DataFrame. The qso_toolbox modules natively work with pandas dataframes. However, as long as the tables are either  an astropy record array or a fits record array, they will be converted inside the modules.


In [2]:
import pandas as pd 
from astropy.io import fits 
from astropy.table import Table

df_hdf = pd.read_hdf('./data/stripe82_milliquas_190210.hdf5',key='data')
print(df_hdf[:10])

df_csv = pd.read_csv('./data/stripe82_milliquas_190210.csv')
print(df_csv[:10])

hdu = fits.open('./data/stripe82_milliquas_190210.fits')
t = Table(hdu[1].data)
df_fits = t.to_pandas()
print(df_fits[:10])

      mq_ra    mq_dec                   mq_name   mq_z mq_cite
0  0.006464 -0.559066  SDSS J000001.55-003332.6  2.219   DR14Q
1  0.006855  0.583184  SDSS J000001.64+003459.4  2.271   DR14Q
2  0.008067 -0.240971  SDSS J000001.93-001427.4  2.163   DR14Q
3  0.009470  0.627545  SDSS J000002.27+003739.1  0.248    DR14
4  0.013229  1.252967  SDSS J000003.17+011510.6  2.354   DR14Q
5  0.013630  0.192449  SDSS J000003.27+001132.8  1.485   DR14Q
6  0.019943 -0.909635  SDSS J000004.78-005434.6  0.858   DR14Q
7  0.020696 -0.278343  SDSS J000004.96-001642.0  2.879   DR14Q
8  0.020929 -0.641393  SDSS J000005.02-003829.0  2.662   DR14Q
9  0.022563  0.443903  SDSS J000005.41+002638.0  0.928   DR14Q
      mq_ra    mq_dec                   mq_name   mq_z mq_cite
0  0.006464 -0.559066  SDSS J000001.55-003332.6  2.219   DR14Q
1  0.006855  0.583184  SDSS J000001.64+003459.4  2.271   DR14Q
2  0.008067 -0.240971  SDSS J000001.93-001427.4  2.163   DR14Q
3  0.009470  0.627545  SDSS J000002.27+003739.1  0.248 

## 1) Downloading a sample of cutouts 

The function inside the catalog_tools.py that is responsible for downloading image cutouts is called "catalog_tools.get_photometry()".

Here is the beginning of the function definition for an overview:

def get_photometry(table, ra_col_name, dec_col_name, surveys, bands, image_path,
                   fovs,
                   verbosity=0):
    
    """Download photometric images for all objects in the given input table.

    Lists need to be supplied to specify the survey, the photometric passband
    and the field of fiew. Each entry in the survey list corresponds to one
    entry in the passband and field of view lists.
    
    The downloaded images will be save in the image_path directory using a
    unique filename based on the target position, survey + data release,
    passband and field of view.

    Image name
    [Epoch Identifier][RA in HHMMSS.SS][DEC in DDMMSS.SS]_
                                 [SURVEY]_[PASSBAND]_fov[FIELD OF VIEW].fits
    
    A example for DES DR1 z-band with a field of view of 100 arcsec:
    J224029.28-000511.83_desdr1_z_fov100.fits

    The list of field of views will be accurately downloaded for desdr1. For
    the download of the unWISE image cutouts the field of views will be
    converted to number of pixels with npix = fov / 60. /4. * 100, with an
    upper limit of 256 pixels.
    
    IMPORTANT:
    The function will skip downloads for targets with exactly the same 
    specifications (filenames) that already exist in the folder.

    :param table: table object
        Input data table with at least RA and Decl. columns
    :param ra_col_name: string
        Exact string for the RA column in the table
    :param dec_col_name: string
        Exact string for the Decl. column in the table
    :param surveys: list of strings
        List of survey names, length has to be equal to bands and fovs
    :param bands: list of strings
        List of band names, length has to be equal to surveys and fovs
    :param image_path: string
        Path to the directory where all the images will be stored
    :param fovs: list of floats
        Field of view in arcseconds, length has be equal to surveys and bands
    :param verbosity:
        Verbosity > 0 will print verbose statements during the execution
    :return: None
    """

We will now create a 'cutouts' directory in this folder, where the cutouts will be saved.

In [3]:
! mkdir cutouts 
! ls 

mkdir: cutouts: File exists
GettingStarted.ipynb                  quasat_test_vhsdownloadcutout.csv
HowTo-1-DownloadingImageCutouts.ipynb test.fits
Test Catutils.ipynb                   test.png
Test multiprocessing.ipynb            test_download100.csv
TestUnWISE image download.ipynb       test_unwise-allwise_w1_npix100.fits
Untitled.ipynb                        tmp.tar.gz
cutouts                               vhs_test.fits
data                                  vhs_wget_file_test.wget
mq_test.fits                          vsa_test_upload.csv
quasar_test_set.csv


We first choose a subset of Stripe 82 to make sure not to download over 35000 cutouts per band.

In [4]:
test_set = df_hdf.query('340 < mq_ra < 350 and -1.26 < mq_dec < 0')
print(test_set.shape)

(1344, 5)


Now we can begin to download some image cutouts. We will download DES DR1 z-band and UNWISE W1-band cutouts for a start. Before we can call the function we need to specify the surveys, bands and field of views. We will set verbosity to 2, to see what the code does on it's way.

In [5]:
surveys = ['desdr1','unwise-neo3']
bands = ['z','w1']
fovs = [100, 100]

ct.get_photometry(test_set[:10], 'mq_ra', 'mq_dec', surveys, bands, './cutouts/', fovs, verbosity=2)

/Users/schindler/anaconda3/lib/python3.6/site-packages/qso_toolbox/catalog_tools.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  epoch="J")


## 2) Downloading cutouts using multiprocessing

If the code successfully downloaded the cutouts for these 10 targets, let's do it for another 200. However, in this case we compare the function used above to it's multiprocessing capable version "get_photometry_mp". And for fun we will time the runs of both functions, each downloading 100 cutouts. We will set the number of jobs ('n_jobs') to 5. Please change this depending on your CPU capabilities. We further set verbosity to 0, to not be overrun by internal messages.

In [6]:
import time 

# Normal run 
start_time = time.time()
ct.get_photometry(test_set[10:110], 'mq_ra', 'mq_dec', surveys, bands, './cutouts/', fovs, verbosity=2)
print("Elapsed time: {0:.2f} sec" .format(
                                time.time() - start_time))

# Multiprocessing run 
start_time = time.time()
ct.get_photometry_mp(test_set[110:210], 'mq_ra', 'mq_dec', surveys, bands, './cutouts/', fovs, n_jobs=5, verbosity=2)
print("Elapsed time: {0:.2f} sec" .format(
                                time.time() - start_time))

The full image list contains 36 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224117.82-005015.24_desdr1_z_fov100 unsuccessful


The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224123.51-010224.87_desdr1_z_fov100 unsuccessful


The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224142.49-004541.16_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224143.31-004637.20_desdr1_z_fov100 unsuccessful


The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224145.71-002516.51_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224148.00-005953.97_desdr1_z_fov100 unsuccessful


The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224206.89-005151.85_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224207.07-003948.39_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224211.43-002450.89_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224215.96-005326.27_desdr1_z_fov100 unsuccessful


...

The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224259.09-002512.71_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224301.15-010048.61_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224314.04-005627.26_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224320.41-004918.73_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224320.63-003945.87_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error


The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224344.80-004902.93_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224349.90-003630.02_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224352.33-004516.06_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224355.55-002714.61_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224358.75-002802.60_desdr1_z_fov100 unsuccessful
The full image list contains 18 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error


/Users/schindler/anaconda3/lib/python3.6/site-packages/qso_toolbox/catalog_tools.py:318: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  epoch="J")


The full image list contains 16 entries


The full image list contains 16 entries
downloading deepest stacked image...
The full image list contains 16 entries
The full image list contains 16 entries
downloading deepest stacked image...
downloading deepest stacked image...
downloading deepest stacked image...
The full image list contains 34 entries
downloading deepest stacked image...
HTTP Error 500: Internal Server Error
Download of J224803.17-003821.62_desdr1_z_fov100 unsuccessful
HTTP Error 500: Internal Server Error
Download of J224712.59-002449.32_desdr1_z_fov100 unsuccessful
HTTP Error 500: Internal Server Error
Download of J224739.33-003952.73_desdr1_z_fov100 unsuccessful
HTTP Error 500: Internal Server Error
Download of J224828.16-004109.00_desdr1_z_fov100 unsuccessful
HTTP Error 500: Internal Server Error
Download of J224733.47-002009.18_desdr1_z_fov100 unsuccessful
The full image list contains 16 entries
The full image list contains 16 entries
downloading deepest stacked image...
The full image list contains 16 entrie

You should be able to see a better performance by the multiprocessing version of the get_photometry routine. We can now have a quick look at the files in the folder 

In [7]:
! ls ./cutouts/

J224007.14-005332.81_desdr1_z_fov100.fits
J224007.14-005332.81_unwise-neo3_w1_fov100.fits
J224007.90-011522.10_desdr1_z_fov100.fits
J224007.90-011522.10_unwise-neo3_w1_fov100.fits
J224009.37-003856.60_desdr1_z_fov100.fits
J224009.37-003856.60_unwise-neo3_w1_fov100.fits
J224011.47-011448.62_desdr1_z_fov100.fits
J224011.47-011448.62_unwise-neo3_w1_fov100.fits
J224013.39-005404.40_desdr1_z_fov100.fits
J224013.39-005404.40_unwise-neo3_w1_fov100.fits
J224014.28-001121.15_desdr1_z_fov100.fits
J224014.28-001121.15_unwise-neo3_w1_fov100.fits
J224017.29-011442.85_desdr1_z_fov100.fits
J224017.29-011442.85_unwise-neo3_w1_fov100.fits
J224018.29-005223.75_desdr1_z_fov100.fits
J224018.29-005223.75_unwise-neo3_w1_fov100.fits
J224018.58-000656.89_desdr1_z_fov100.fits
J224018.58-000656.89_unwise-neo3_w1_fov100.fits
J224022.77-000151.97_desdr1_z_fov100.fits
J224022.77-000151.97_unwise-neo3_w1_fov100.fits
J224023.25-003555.74_desdr1_z_fov100.fits
J224023.25-003555.74_unwise-neo3_w1_fov100.fits
J224026.75

### In some cases it is possible that the download will not succeed, because it experiences server errors. For example the infamous "HTTP Error 500: Internal Server Error"

## 3) Downloading image cutouts for subsequent examples
At last we will download image cutouts for a range of surveys and bands. Please make sure you have enough free disk space for this. It will take roughly ??? MB. However, this is necessary to run the other jupyter notebooks, which rely on these image cutouts.